<a href="https://colab.research.google.com/github/donniekramer8/Final-Project/blob/master/new_4_21_AlphaFold2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install dependencies
%%bash -s $use_amber $use_templates $python_version

set -e

USE_AMBER=$1
USE_TEMPLATES=$2
PYTHON_VERSION=$3

if [ ! -f COLABFOLD_READY ]; then
  echo "installing colabfold..."
  # install dependencies
  # We have to use "--no-warn-conflicts" because colab already has a lot preinstalled with requirements different to ours
  pip install -q --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold" "tensorflow-cpu==2.11.0"
  pip uninstall -yq jax jaxlib
  pip install -q "jax[cuda]==0.3.25" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html


  # for debugging
  ln -s /usr/local/lib/python3.*/dist-packages/colabfold colabfold
  ln -s /usr/local/lib/python3.*/dist-packages/alphafold alphafold
  touch COLABFOLD_READY
fi

# setup conda
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    echo "installing conda..."
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniconda3-latest-Linux-x86_64.sh
    conda config --set auto_update_conda false
    touch CONDA_READY
  fi
fi
# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  echo "installing hhsuite..."
  conda install -y -q -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 python="${PYTHON_VERSION}" 2>&1 1>/dev/null
  touch HH_READY
fi
# setup openmm for amber refinement
if [ ${USE_AMBER} == "True" ] && [ ! -f AMBER_READY ]; then
  echo "installing amber..."
  conda install -y -q -c conda-forge openmm=7.5.1 python="${PYTHON_VERSION}" pdbfixer cryptography==38.0.4 2>&1 1>/dev/null
  touch AMBER_READY
fi

installing colabfold...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 2.11.0 which is incompatible.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
tensorflow 2.12.0 requires tensorboard<2.13,>=2.12, but you have tensorboard 2.11.2 which is incompatible.
tensorflow 2.12.0 requires tensorflow-estimator<2.13,>=2.12.0, but you have tensorflow-estimator 2.11.0 which is incompatible.
orbax 0.1.7 requires jax>=0.4.6, but you have jax 0.3.25 which is incompatible.
flax 0.6.8 requires jax>=0.4.2, but you have jax 0.3.25 which is incompatible.
chex 0.1.7 requires jax>=0.4.6, but you have jax 0.3.25 which is incompatible.


In [ ]:
# Do not need to run
# Donnie 4/21: I only noticed this cell after I did everything below, but I figured I will keep it here just in case we want to change below

#@title Package and download results
#@markdown If you are having issues downloading the result archive, try disabling your adblocker and run this cell again. If that fails click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).

if msa_mode == "custom":
  print("Don't forget to cite your custom MSA generation method.")

files.download(f"{jobname}.result.zip")

if save_to_google_drive == True and drive:
  uploaded = drive.CreateFile({'title': f"{jobname}.result.zip"})
  uploaded.SetContentFile(f"{jobname}.result.zip")
  uploaded.Upload()
  print(f"Uploaded {jobname}.result.zip to Google Drive with ID {uploaded.get('id')}")

In [2]:
!pip install biopython
from Bio import SeqIO
from google.colab import drive
import os
import sys

drive.mount('/content/gdrive')

def save_to_drive(jobname, sequence, msa_method):
  # add settings file
  %cd /content/
  !zip -r /content/{jobname}.zip /content/{jobname}/
  %cd /content/gdrive/My\ Drive/
  !mkdir -p ./collab/{jobname}/
  ! mv ../../{jobname}.zip ./collab/{jobname}/
  settings_path = "./collab/{}/settings.txt".format(jobname)

  with open(settings_path, "w") as text_file:
    text_file.write(f"method=RoseTTAFold\n")
    text_file.write(f"sequence={sequence}\n")
    text_file.write(f"msa_method={msa_method}\n")
    text_file.write(f"use_templates=False\n")
  
  !rm -r /content/{jobname}/
  print("Saved job {} to drive".format(jobname))

sequences = SeqIO.parse("all_seqs.fasta", "fasta")

for x in sequences:
  print(x)
  break

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/gdrive
ID: 4FPQ_1|Chains
Name: 4FPQ_1|Chains
Description: 4FPQ_1|Chains A, B, C, D|Avirulence Effector AvrLm4-7|Leptosphaeria maculans (985895)
Number of features: 0
Seq('AYVCREASISGEIRYPQGTCPTKTEALNDCNKVTKGLIDFSQSHQRAWGIDMTA...HHH')


In [35]:
import sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from Bio import BiopythonDeprecationWarning
warnings.simplefilter(action='ignore', category=BiopythonDeprecationWarning)
from pathlib import Path
from colabfold.download import download_alphafold_params, default_data_dir
from colabfold.utils import setup_logging
from colabfold.batch import get_queries, run, set_model_type
from colabfold.plot import plot_msa_v2
import os
import numpy as np
from colabfold.colabfold import plot_protein
from pathlib import Path

msa_mode = "mmseqs2_uniref_env"
pair_mode = "unpaired_paired"
# if "mmseqs2" in msa_mode:
#   a3m_file = os.path.join(jobname,f"{jobname}.a3m")
model_type = "auto"
num_recycles = None
recycle_early_stop_tolerance = None
num_seeds = 1
use_dropout = False
dpi = 200
max_msa = None
num_relax = 0

# Probably need to change
use_templates = False
custom_template_path = None

#### Save settings
save_all = False
save_recycles = False
save_to_google_drive = False

display_images = False

def prediction_callback(protein_obj, length, prediction_result, input_features, mode):
  model_name, relaxed = mode
  if not relaxed:
    if display_images:
      fig = plot_protein(protein_obj, Ls=length, dpi=150)
      plt.show()
      plt.close()

%cd /content/gdrive/MyDrive/StructBio
download_alphafold_params(model_type, Path("."))

/content/gdrive/MyDrive/StructBio


In [36]:
max = 500
count = 0

for seq_record in sequences:
  if count < max:
    sequence = str(seq_record.seq)
    query_sequence = sequence.translate(str.maketrans('', '', ' \n\t')).upper()
    sequence_id =  seq_record.id.split("_")[0]
    jobname = "AF_" + sequence_id
    print(jobname)
    print("Starting: {}".format(jobname))
    if len(sequence) < 500:
      %cd /content/gdrive/MyDrive/StructBio
      # my path is named "StructBio", change to fit yours
      if not (os.path.exists("./StructBio/{}".format(jobname))):
        print("New prediction")

        a3m_file = os.path.join(jobname,f"{jobname}.a3m")
        # make directory to save results
        os.makedirs(jobname, exist_ok=True)
        queries_path = os.path.join(jobname, f"{jobname}.csv")
        result_dir = jobname

        with open(queries_path, "w") as text_file:
          text_file.write(f"id,sequence\n{jobname},{query_sequence}")
        queries, is_complex = get_queries(queries_path)
        model_type = set_model_type(is_complex, model_type)

        if "multimer" in model_type and max_msa is not None:
          use_cluster_profile = False
        else:
          use_cluster_profile = True

        download_alphafold_params(model_type, Path("."))
        results = run(
            queries=queries,
            result_dir=result_dir,
            use_templates=use_templates,
            custom_template_path=custom_template_path,
            num_relax=num_relax,
            msa_mode=msa_mode,    
            model_type=model_type,
            num_models=5,
            num_recycles=num_recycles,
            recycle_early_stop_tolerance=recycle_early_stop_tolerance,
            num_seeds=num_seeds,
            use_dropout=use_dropout,
            model_order=[1,2,3,4,5],
            is_complex=is_complex,
            data_dir=Path("."),
            keep_existing_results=False,
            rank_by="auto",
            pair_mode=pair_mode,
            stop_at_score=float(100),
            prediction_callback=prediction_callback,
            dpi=dpi,
            zip_results=False,
            save_all=save_all,
            max_msa=max_msa,
            use_cluster_profile=use_cluster_profile,
            # input_features_callback=input_features_callback,
            save_recycles=save_recycles,
            )
        results_zip = f"{jobname}.result.zip"
        os.system(f"zip -r {results_zip} {jobname}")
      else:
        print("This prediction ({}) already exists on file: skipping".format(jobname))
        count += 1 
    else:
      print("Protein {} is too long".format(jobname))
  else:
    break

AF_7FT2
Starting: AF_7FT2
/content/gdrive/MyDrive/StructBio
New prediction


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:02 remaining: 00:00]


AF_7FT3
Starting: AF_7FT3
/content/gdrive/MyDrive/StructBio
New prediction


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:02 remaining: 00:00]


KeyboardInterrupt: ignored